In [27]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine, MetaData, text
from smartsheet_dataframe import get_as_df

In [28]:
# Storing Smartsheet information in variables
shitz_token = "wMvGniITjDFd7ClZPE44wtJGvZkM7Hg7mP7if" #Bily's personal token to access G-A smartsheet data. Anyone can obtain their own token
mssid = 3005704744265604 # The ID number that references the MSS sheet

In [29]:
mss_df = get_as_df(type_='sheet',
                  token=shitz_token,
                  id_=mssid,
                  )

In [30]:
mss_df.head()

,row_id,parent_id,IntakeID,FolderID,UniqueID,TicketID,Tract,CMP,Stand,Acres,...,Activity Implemented,Applied GA_ACRES,PstBA,PstTPA,PstGTA,RP Completed,Notes,Completed Date,Modified,Modified By
0,6438718047686532,,WRR240404,WRR240404,WRR240404TH02,WRR240404TH02,Mayhaw,122-1,15,55.0,...,,,,,,,,,2024-04-08T13:20:55Z,
1,8918288567013252,,WRR240404,WRR240404,WRR240404TH01,WRR240404TH01,Mayhaw,122-1,1,23.01,...,,,,,,,,,2024-04-08T13:20:55Z,
2,3313610417934212,,WRR240325,WRR240325,WRR240325BN01,WRR240325BN01,Telogia,18.0,"1, 2, 3, 5, 9, 13, 15",240.1,...,,,,,,,,,2024-04-03T19:32:47Z,
3,8625293074009988,,WRR240318,WRR240318,WRR240318BN02,WRR240318BN02,Telogia,"42, 43, 44","6, 7, 9, 1, 3, 4, 8",1493.15,...,,,,,,,,,2024-04-03T19:32:47Z,
4,7021399277473668,,WRR240318,WRR240318,WRR240318BN01,WRR240318BN01,Telogia,"18, 20","6, 7, 12, 11",365.07,...,,,,,,,,,2024-04-03T19:32:47Z,


In [31]:
mss_df.columns

Index(['row_id', 'parent_id', 'IntakeID', 'FolderID', 'UniqueID', 'TicketID',
       'Tract', 'CMP', 'Stand', 'Acres', 'Age', 'GA_ACRES', 'County', 'State',
       'Plot Numbers', 'Status', 'Guidance', 'Expiration', 'GuidanceProvider',
       'Scheduled', 'Stand Type', 'Origin', 'PreBA', 'PreTPA', 'PreGTA',
       'PlnBA', 'PlnTPA', 'PlnGTA', 'Client GT', 'GA GT',
       'Current RP Expected GT', 'Future GT', 'Total GT', 'Timeframe',
       'Reporting Period', 'Activity Implemented', 'Applied GA_ACRES', 'PstBA',
       'PstTPA', 'PstGTA', 'RP Completed', 'Notes', 'Completed Date',
       'Modified', 'Modified By'],
      dtype='object')

In [32]:
len(mss_df.isna())

586

In [33]:
len(mss_df)

586

In [34]:
# Grab just the needed columns
client_preimp = mss_df[['FolderID',
                        'UniqueID',
                        'Tract',
                        'CMP',
                        'Stand',
                        'Acres',
                        'Age',
                        'County',
                        'State',
                        'Stand Type',
                        'Origin',
                        'PreBA',
                        'PreTPA',
                        'PreGTA',
                        'PlnBA',
                        'PlnTPA',
                        'PlnGTA',
                        'Client GT',
                        'Future GT',
                        'Reporting Period',
                        'Notes']].copy()
client_preimp = client_preimp.rename(columns={'FolderID':'folderid',
                                              'UniqueID':'uniqueid',
                                              'Tract': 'tract',
                                              'CMP': 'comp',
                                              'Stand': 'stand',
                                              'Acres': 'client_acres',
                                              'Age': 'age',
                                              'County': 'county',
                                              'State': 'state',
                                              'Stand Type': 'species',
                                              'Origin': 'origin',
                                              'PreBA': 'pre_ba',
                                              'PreTPA': 'pre_tpa',
                                              'PreGTA': 'pre_gta',
                                              'PlnBA': 'planned_ba',
                                              'PlnTPA': 'planned_tpa',
                                              'PlnGTA': 'planned_gta',
                                              'Client GT': 'client_gt_current',
                                              'Future GT': 'client_gt_future',
                                              'Reporting Period': 'rp_requested',
                                              'Notes': 'notes'})

In [35]:
# Take out non-harvest activities
non_activity = ['BN', 'MW', 'SP', 'FT', 'UE', 'WP']
client_preimp = client_preimp.loc[~client_preimp['uniqueid'].str.contains('|'.join(non_activity), case=False, na=False)]

In [36]:
client_preimp[client_preimp.isna().any(axis=1)]

,folderid,uniqueid,tract,comp,stand,client_acres,age,county,state,species,...,pre_ba,pre_tpa,pre_gta,planned_ba,planned_tpa,planned_gta,client_gt_current,client_gt_future,rp_requested,notes


In [37]:
client_preimp['client_acres'] = pd.to_numeric(client_preimp['client_acres'], errors='coerce')
client_preimp['age'] = pd.to_numeric(client_preimp['age'], errors='coerce')
client_preimp['pre_ba'] = pd.to_numeric(client_preimp['pre_ba'], errors='coerce')
client_preimp['pre_tpa'] = pd.to_numeric(client_preimp['pre_tpa'], errors='coerce')
client_preimp['pre_gta'] = pd.to_numeric(client_preimp['pre_gta'], errors='coerce')
client_preimp['planned_ba'] = pd.to_numeric(client_preimp['planned_ba'], errors='coerce')
client_preimp['planned_tpa'] = pd.to_numeric(client_preimp['planned_tpa'], errors='coerce')
client_preimp['planned_gta'] = pd.to_numeric(client_preimp['planned_gta'], errors='coerce')
client_preimp['client_gt_current'] = pd.to_numeric(client_preimp['client_gt_current'], errors='coerce')
client_preimp['client_gt_future'] = pd.to_numeric(client_preimp['client_gt_future'], errors='coerce')
client_preimp['rp_requested'] = pd.to_numeric(client_preimp['rp_requested'], errors='coerce')

In [38]:
# set null values to -1
client_preimp = client_preimp.fillna(-1)

In [39]:
# setup db connection
db_host = 'localhost'
db_name = 'HarvestManagementTrial'
db_user = 'postgres'
db_password = 'root'

connection_string = f'postgresql://{db_user}:{db_password}@{db_host}/{db_name}'

# Create the SQLAlchemy engine
engine = create_engine(connection_string)

In [40]:
client_preimp.to_sql('client_table_preimp', engine, if_exists='append', index=False)

497

In [42]:
engine.dispose()